In [2]:
import torch
if torch.cuda.is_available():
    import torch.cuda as t
else:
    import torch as t
    
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
torch.cuda.is_available()

True

In [4]:
import json, os
from pathlib import Path

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self, n_classes=2, kernels=4, layers=1, max_pool_size=0):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential()
        in_channels = 1
        out_channels = kernels
        for n in range(layers):
            self.conv.add_module('conv'+str(n), nn.Conv2d(in_channels, out_channels, 3))
            in_channels = out_channels
            out_channels += 2
        self.maxpool = nn.MaxPool2d(max_pool_size)
        self.dropout = nn.Dropout()
        self.out_dim = (out_channels -2) * ((28-layers * 2 )**2)
        
        self.fc1 = nn.Linear(self.out_dim,100)
        self.fc2 = nn.Linear(100, n_classes)
            
        
    def forward(self, x):
        x = self.conv(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(-1,self.out_dim)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [5]:
class CNN(nn.Module):
    def __init__(self, n_classes=2):
        super(CNN, self).__init__()
        self.conv = nn.Sequential()
        self.conv.add_module('conv1', nn.Conv2d(1, 32, 3))
        self.conv.add_module('relu', nn.ReLU())
        self.conv.add_module('max1', nn.MaxPool2d(2))
        self.conv.add_module('conv2', nn.Conv2d(32, 64, 3))
        self.conv.add_module('relu2', nn.ReLU())
        self.conv.add_module('max2', nn.MaxPool2d(2))
        self.conv.add_module('conv3', nn.Conv2d(64, 64, 3))
        self.conv.add_module('relu3', nn.ReLU())
        
        self.fc1 = nn.Linear(576,64)
        self.fc2 = nn.Linear(64, n_classes)
            
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1,576)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [6]:
# s = nn.Sequential()
# net.conv.add_module('conv2', nconv)

In [6]:
net = CNN(n_classes=10).cuda()
net.conv.children

<bound method Module.children of Sequential(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (max1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (max2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
)>

In [7]:

# for data in testloader:
#     images, labels = data
net = SimpleCNN(n_classes=10,layers=2,kernels=10)
net.conv.children

<bound method Module.children of Sequential(
  (conv0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(10, 12, kernel_size=(3, 3), stride=(1, 1))
)>

In [8]:
config = json.load(open(os.path.expanduser("~/.thesis.conf")))
root = Path(config['datasets']) / Path("MNIST/") 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
mnist_set = torchvision.datasets.MNIST(str(root),train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(mnist_set, batch_size=64,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(str(root), train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

dataiter = iter(trainloader)
images, label = dataiter.next()

In [11]:
type(mnist_set)

torchvision.datasets.mnist.MNIST

In [10]:
# net = SimpleCNN(10,layers=3)#.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9)
# loss_

In [34]:

for epoch in range(0):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [14]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    imgs = Variable(images,requires_grad=False).cuda()
    outputs = net(imgs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 10 %


In [9]:
from torch.utils.trainer import Trainer
from torch.utils.trainer.plugins import Logger, LossMonitor, TimeMonitor, ProgressMonitor

In [10]:
# net = SimpleCNN(n_classes=10,layers=2,kernels=10)#.cuda()
net = CNN(n_classes=10).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(),lr=0.001, alpha=0.9,weight_decay=0.0)

trainer = Trainer(net, criterion, optimizer, testloader)
trainer.register_plugin(LossMonitor())
trainer.register_plugin(TimeMonitor())
trainer.register_plugin(ProgressMonitor())

# trainer.register_plugin(LossSave())
trainer.register_plugin(Logger(['loss', 'time'],interval=[(500, 'iteration'), (1, 'epoch')]))

In [11]:
trainer.run(epochs=1)

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.FloatTensor for argument #2 'weight'

In [17]:
trainer.model

CNN(
  (conv): Sequential(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (relu): ReLU()
    (max1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu2): ReLU()
    (max2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu3): ReLU()
  )
  (fc1): Linear(in_features=576, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [14]:
class LossSave(LossMonitor):
    stat_name = 'loss'
    losslog = []
    
    def _get_value(self, iteration, input, target, output, loss):
        self.losslog.append((iteration, loss))
        return loss[0]